In [1]:
import csv
import json
import mysql.connector

# please modify hw4_config.json to your own config file
with open('hw4_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']


## 1-1: Input data into student table

In [2]:
mydb = mysql.connector.connect(
  host = host,user = user, password = passwd, 
    auth_plugin='mysql_native_password' )

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE DB_class")
mycursor.execute("USE DB_class")

mycursor.execute("CREATE TABLE STUDENT (身份 varchar(20), 系所 varchar(225), 年級 varchar(10), 學號 varchar(225) primary key NOT NULL, 姓名 varchar(225) NOT NULL, 信箱 varchar(225), 班別 varchar(225))")

with open(dbms_student_path,mode = 'r') as file:
    input_data = csv.reader(file)
    next(input_data)
    insert = "INSERT INTO STUDENT (身份,系所,年級,學號,姓名,信箱,班別) VALUES (%s,%s,%s,%s,%s,%s,%s)"
    for row in input_data:
        mycursor.execute(insert, tuple(row))

#mycursor.execute('SELECT * FROM STUDENT')
#output = mycursor.fetchall()
#print(output)

## 1-2: Read the information about yourself

In [3]:
mycursor.execute('SELECT * FROM STUDENT WHERE 學號 = "R10455001"')

output = mycursor.fetchall()
print(output)


[('學生', '基蛋所', '2', 'R10455001', '陳品瑄 (PIN-XUAN CHEN)', 'r10455001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-3: Update your own value of the "身份" column 

In [4]:
mycursor.execute('UPDATE STUDENT SET 身份 = "優等生" WHERE 學號 = "R10455001" ')
mycursor.execute('SELECT * FROM STUDENT WHERE 學號 = "R10455001"')

output = mycursor.fetchall()
print(output)

[('優等生', '基蛋所', '2', 'R10455001', '陳品瑄 (PIN-XUAN CHEN)', 'r10455001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-4: Insert three new students 

In [5]:
with open (new_student_path, mode = 'r') as file:
    new_data = csv.reader(file)
    next(new_data)
    insert = "INSERT INTO STUDENT (身份,系所,年級,學號,姓名,信箱,班別) VALUES (%s,%s,%s,%s,%s,%s,%s)"
    for row in new_data:
        mycursor.execute(insert, tuple(row))

mycursor.execute('SELECT * FROM STUDENT WHERE 學號 = "R10123456" OR 學號 = "B09987653" OR 學號 = "R11123001" ')

output = mycursor.fetchall()
print(output)

[('學生', '物理系', '3', 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)'), ('旁聽生', '電機所', '2', 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)'), ('觀察者', '電信所', '1', 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-5: Prepared select statement 

In [6]:
for i in ["R10123456", "B09987653", "R11123001" ]:
    mycursor.execute('SELECT * FROM STUDENT WHERE 學號 = %s',(i,))
    output = mycursor.fetchall()
    print(output)

[('旁聽生', '電機所', '2', 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]
[('學生', '物理系', '3', 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]
[('觀察者', '電信所', '1', 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')]


## 1-6: Group information

In [7]:
mycursor.execute('ALTER TABLE STUDENT ADD final_group varchar(10) DEFAULT "0" ')
mycursor.execute('ALTER TABLE STUDENT ADD final_captain varchar(10) DEFAULT "N" ')



#change group
for i in ["F08921a05", "D06921008", "R11631028", "R08424026", "R10455001" ]:
    mycursor.execute('UPDATE STUDENT SET final_group = "5" WHERE 學號 = %s', (i,))
    
#change captain 
mycursor.execute('UPDATE STUDENT SET final_captain = "Y" WHERE 學號 = "F08921a05" ')

#output our group information
mycursor.execute('SELECT * FROM STUDENT WHERE final_group = "5" ')
output = mycursor.fetchall()
print(output)


[('學生', '電機系', '6', 'D06921008', '黃尹姿 (HUANG YIN TZU)', 'd06921008@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', '5', 'N'), ('學生', '電機系', '3', 'F08921A05', '陳憶賢 (YI-HSIEN CHEN)', 'f08921a05@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', '5', 'Y'), ('學生', '基蛋所', '3', 'R08424026', '莊惠文 (CHUANG HUI WEN)', 'r08424026@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', '5', 'N'), ('優等生', '基蛋所', '2', 'R10455001', '陳品瑄 (PIN-XUAN CHEN)', 'r10455001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', '5', 'N'), ('學生', '生機系', '1', 'R11631028', '劉易霖 (LIU, YIH-LIN)', 'r11631028@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', '5', 'N')]


In [8]:
mydb.commit()
mydb.close()
mycursor.close()

True